In [1]:
import pandas as pd
import numpy as np
import os
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut
from geopy.extra.rate_limiter import RateLimiter
from time import sleep

####### OPTION 1 - Set File Path to Google Drive (You may need to install google drive app on your machine for this to work) #########
google_drive_path = '/Users/tejveero/Library/CloudStorage/GoogleDrive-tejveerobr@gmail.com/My Drive/SCM.800/Ingestion/raw_files(DO_NOT_EDIT)'

In [ ]:
# Ingest the Excel and CSV files
df_cogs = pd.read_excel(os.path.join(google_drive_path, 'Cost File (MIT) .XLSX'))
df_cost_parcel = pd.read_csv(os.path.join(google_drive_path, 'GENE_2024_FEDEX_UPS.csv'))
df_cost_tl = pd.read_excel(os.path.join(google_drive_path, 'ShipmentLevelDetails_RXO2024.xlsx'))
df_emissions = pd.read_excel(os.path.join(google_drive_path, 'MIT_Copy of XPO_RXO input CO2_v2_2020_2 - Eco_transit_data (1)_OUT_258.xlsx'))
df_customers = pd.read_excel(os.path.join(google_drive_path, 'Z004 Ship to party 20241017.xlsx')) 

#df_wh_capacity - create manually from picture

#Edits needed
df_sku_list = pd.read_excel(os.path.join(google_drive_path, 'MIT info 07FEB2025.xlsx')) 
df_demand1 = pd.read_excel(os.path.join(google_drive_path, '8 week deliveries from KDC-HDC 18OCT2024.xlsx'))
df_demand2 = pd.read_excel(os.path.join(google_drive_path, 'UPS 8 Weeks shipments.xlsx'))
df_x = pd.read_excel(os.path.join(google_drive_path, 'Unrestricted Inventory 16OCT2024.xlsx')) 
df_packaging_site = pd.read_excel(os.path.join(google_drive_path, 'Packaging Site.xlsx'))

#Not Used
df_pallet_case_qty = pd.read_excel(os.path.join(google_drive_path, 'Pallet and Case Qty 24NOV2024.XLSX'))
df_product_class_matrix = pd.read_excel(os.path.join(google_drive_path, 'Marketed Product Class Matrix.xlsx'))
df_demand = pd.read_excel(os.path.join(google_drive_path, 'KDc-HDC Strategy 15OCT2024.xlsx'))
df_kdc_to_ups_shipments = pd.read_excel(os.path.join(google_drive_path, 'KDC to UPS 8 Week 24NOV2024.XLSX'))
df_ups_to_kdc_shipments = pd.read_excel(os.path.join(google_drive_path, 'UPS to KDC 8 Week.XLSX'))
df_zmm_specific = pd.read_excel(os.path.join(google_drive_path, 'ZMM 229 specific 19OCT2024 v2.xlsx')) #Temp conditions

/var/folders/h6/jwpn2tnn5yn3hvc9dl8zhrq40000gn/T/ipykernel_9329/1459958944.py:3: DtypeWarning: Columns (3,9) have mixed types. Specify dtype option on import or set low_memory=False.
  df_cost_parcel = pd.read_csv(os.path.join(google_drive_path, 'GENE_2024_FEDEX_UPS.csv'))


Creating a list of files and fields for data catalogue

In [3]:
# List of DataFrames and their source file names
dataframes_info = {
    'Cost File (MIT) .XLSX': df_cogs,
    'GENE_2024_FEDEX_UPS.csv': df_cost_parcel,
    'ShipmentLevelDetails_RXO2024.xlsx': df_cost_tl,
    'MIT_Copy of XPO_RXO input CO2_v2_2020_2 - Eco_transit_data (1)_OUT_258.xlsx': df_emissions,
    'Z004 Ship to party 20241017.xlsx': df_customers,
    'MIT info 07FEB2025.xlsx': df_sku_list,
    '8 week deliveries from KDC-HDC 18OCT2024.xlsx': df_demand1,
    'UPS 8 Weeks shipments.xlsx': df_demand2,
    'Unrestricted Inventory 16OCT2024.xlsx': df_x,
    'Packaging Site.xlsx': df_packaging_site,
    'Pallet and Case Qty 24NOV2024.XLSX': df_pallet_case_qty,
    'Marketed Product Class Matrix.xlsx': df_product_class_matrix,
    'KDc-HDC Strategy 15OCT2024.xlsx': df_demand,
    'KDC to UPS 8 Week 24NOV2024.XLSX': df_kdc_to_ups_shipments,
    'UPS to KDC 8 Week.XLSX': df_ups_to_kdc_shipments,
    'ZMM 229 specific 19OCT2024 v2.xlsx': df_zmm_specific
}

# Create a list to hold the summary information
summary_data = []

for file_name, df in dataframes_info.items():
    fields = ", ".join(str(col) for col in df.columns)  # FIX: Ensure all column names are strings
    summary_data.append({
        'Source File Name': file_name,
        'List of Fields': fields,
        'Notes 1': '',   # Empty column
        'Notes 2': ''    # Empty column
    })

# Create the summary DataFrame
df_summary = pd.DataFrame(summary_data)
df_summary

# (Optional) Save to Excel
#df_summary.to_excel('file_field_summary.xlsx', index=False)

,Source File Name,List of Fields,Notes 1,Notes 2
0,Cost File (MIT) .XLSX,"Material, Material Type, Material Description,...",,
1,GENE_2024_FEDEX_UPS.csv,"SHIPMENT_DATE, SSI_CUST_CODE, INVOICE_DATE, WA...",,
2,ShipmentLevelDetails_RXO2024.xlsx,"StartDateLocal, Date Hierarchy - Year, Date Hi...",,
3,MIT_Copy of XPO_RXO input CO2_v2_2020_2 - Eco_...,"line_number, sid, error_type, error_code, erro...",,
4,Z004 Ship to party 20241017.xlsx,"Customer, Title, Name 1, Name 2, Street, City,...",,
5,MIT info 07FEB2025.xlsx,"MDMS Mat. ProdFam, EAN/NDC, Material, Material.1",,
6,8 week deliveries from KDC-HDC 18OCT2024.xlsx,"Delivery, Ship-To Party, Ship-To Party.1, Stre...",,
7,UPS 8 Weeks shipments.xlsx,"Delivery, Route, Item, Ship-to party, Material...",,
8,Unrestricted Inventory 16OCT2024.xlsx,Marcus Test - Unrestricted Inventory and Value...,,
9,Packaging Site.xlsx,"Material Number, Product Description, Shipped ...",,


In [4]:
#Read output dfs
df_m_customer = pd.read_csv(os.path.join(google_drive_path, 'm_customers.csv'))
df_m_demand_wh = pd.read_csv(os.path.join(google_drive_path, 'm_demand_wh.csv'))
df_m_demand = pd.read_csv(os.path.join(google_drive_path, 'm_demand.csv'))
df_m_inbound = pd.read_csv(os.path.join(google_drive_path, 'm_inbound.csv'))
df_m_inventory = pd.read_csv(os.path.join(google_drive_path, 'm_inventory.csv'))
df_m_materials = pd.read_csv(os.path.join(google_drive_path, 'm_materials.csv'))
df_m_warehouses = pd.read_csv(os.path.join(google_drive_path, 'm_warehouses.csv'))

In [5]:
# List of new DataFrames and their source file names
new_dataframes_info = {
    'm_customers.csv': df_m_customer,
    'm_demand_wh.csv': df_m_demand_wh,
    'm_demand.csv': df_m_demand,
    'm_inbound.csv': df_m_inbound,
    'm_inventory.csv': df_m_inventory,
    'm_materials.csv': df_m_materials,
    'm_warehouses.csv': df_m_warehouses
}

# Create a list to hold the new summary information
new_summary_data = []

for file_name, df in new_dataframes_info.items():
    fields = ", ".join(str(col) for col in df.columns)  # convert all column names to string
    new_summary_data.append({
        'Source File Name': file_name,
        'List of Fields': fields,
    })

# Create the new summary DataFrame
df_new_summary = pd.DataFrame(new_summary_data)
df_new_summary

#(Optional) Save to Excel
#df_new_summary.to_excel('output_files_field_summary.xlsx', index=False)

,Source File Name,List of Fields
0,m_customers.csv,"customer_id, customer_name, address, city, sta..."
1,m_demand_wh.csv,"Warehouse, Material, Value"
2,m_demand.csv,"customer_id, material_id, demand"
3,m_inbound.csv,"material_id, Origin, quantity, month"
4,m_inventory.csv,"material_id, warehouse, inventory"
5,m_materials.csv,"material_id, material_name, product, t_zone, m..."
6,m_warehouses.csv,"warehouse_id, warehouse_name, t_zone, lat, lon..."


Creating m_demand

In [15]:
df1 = df_demand1.copy()  # Clean and prepare df_demand1 (KDC/HDC 8-week demand)
df1.columns = [col.strip() for col in df1.columns]  # Fix column names for easier reference

# Rename 'Unnamed: 15' to 't_mode' if it exists
if 'Unnamed: 15' in df1.columns:
    df1 = df1.rename(columns={'Unnamed: 15': 't_mode'})
else:
    df1['t_mode'] = ''  # Add empty column if it doesn't exist

# Extract and rename columns from df1
df1_cleaned = pd.DataFrame({
    'customer_id': df1['Ship-To Party'].astype(str),
    'material_id': df1['Material'].astype(str),
    'demand': df1['Act.delivery qty'].astype(str).str.extract(r'(\d+\.?\d*)')[0].astype(float),
    'historically_served_from': df1['Plant'].astype(str).str.strip().str.upper().map({
        '1013': 'KDC',
        '1012': 'HDC'
    }).fillna('Unknown'),
    't_mode': df1['t_mode'].astype(str)
})

# Clean and prepare df_demand2 (UPS 8-week demand)
df2 = df_demand2.copy()
df2.columns = [col.strip() for col in df2.columns]

# Extract and rename columns from df2
df2_cleaned = pd.DataFrame({
    'customer_id': df2['Ship-to party'].astype(str),
    'material_id': df2['Material'].astype(str),
    'demand': pd.to_numeric(df2['Delivery quantity'], errors='coerce'),
    'historically_served_from': 'UPS',
    't_mode': ''  # df2 doesn't have this column, fill with empty string
})

# Combine both cleaned datasets
m_demand_raw = pd.concat([df1_cleaned, df2_cleaned], ignore_index=True)

# Drop rows with missing demand
m_demand_raw = m_demand_raw.dropna(subset=['demand'])
m_demand_raw

#(Optional) Save to Excel
m_demand_raw.to_excel('m_demand_v2(raw).xlsx', index=False)

In [20]:
m_demand_raw['historically_served_from'].value_counts()

historically_served_from
KDC    13059
HDC     6053
UPS      899
Name: count, dtype: int64

In [ ]:
#Aggregate demand by customer_id and material_id
agg_demand = m_demand_raw.groupby(['customer_id', 'material_id'], as_index=False).agg({
    'demand': 'sum'
})
#Get the mode (i.e. highest occuring value) of historically_served_from 
    # and t_mode per customer_id
mode_info = (
    m_demand_raw
    .groupby('customer_id')
    .agg({
        'historically_served_from': lambda x: x.mode().iloc[0] if not x.mode().empty else 'Unknown',
        't_mode': lambda x: x.mode().iloc[0] if not x.mode().empty else 'Unknown'
    })
    .reset_index()
)
# Merge mode info back into aggregated demand
m_demand = agg_demand.merge(mode_info, on='customer_id', how='left')
m_demand

#Divide 8-week demand by 2 to get monthly demand
m_demand['demand'] = m_demand['demand'] / 2
m_demand

#(Optional) Save to Excel
#m_demand.to_excel('m_demand_v2.xlsx', index=False)

In [21]:
m_demand['historically_served_from'].value_counts()

historically_served_from
KDC    4142
HDC    2396
UPS     136
Name: count, dtype: int64

Creating m_customers

In [9]:
# Create m_customers from df_customers
m_customers = pd.DataFrame({
    'customer_id': df_customers['Customer'].astype(str),
    'customer_name': df_customers['Name 1'].fillna(''),
    'address': df_customers['Street'].fillna(''),
    'city': df_customers['City'].fillna(''),
    'state': df_customers['Rg'].fillna(''),
    'country': df_customers['Cty'].fillna(''),
    'zipcode': df_customers['PostalCode'].astype(str).str[:5],
    't_mode': ''  # initialize with blank
})

########## Join lat/lon from df_m_customer ############
m_customers['customer_id'] = m_customers['customer_id'].astype(str)
df_m_customer['customer_id'] = df_m_customer['customer_id'].astype(str)

m_customers = m_customers.drop(columns=['lat', 'lon'], errors='ignore')
m_customers = m_customers.merge(
    df_m_customer[['customer_id', 'lat', 'lon']],
    on='customer_id',
    how='left'
)

########## Join t_mode from m_demand ##########
m_demand['customer_id'] = m_demand['customer_id'].astype(str)

# Create most common t_mode per customer
t_mode_map = (
    m_demand.groupby('customer_id')['t_mode']
    .agg(lambda x: x.mode().iloc[0] if not x.mode().empty else '')
    .reset_index()
)

# Merge into m_customers with suffix to avoid overwriting directly
m_customers = m_customers.merge(t_mode_map, on='customer_id', how='left', suffixes=('', '_from_demand'))
# Fill only where t_mode is currently blank
m_customers.loc[m_customers['t_mode'] == '', 't_mode'] = m_customers['t_mode_from_demand']
m_customers = m_customers.drop(columns=['t_mode_from_demand']) 
m_customers

,customer_id,customer_name,address,city,state,country,zipcode,t_mode,lat,lon
0,125,"DAKOTA DRUG, INC",4121 12TH AVE NW,FARGO,ND,US,58102,Fedex Next Day AM,46.927094,-96.834703
1,178,Prescription Supply Inc.,2233 Tracy Road,Northwood,OH,US,43619,NaN,41.603711,-83.470490
2,319,Walgreens Specialty Pharmacy,130 Enterprise Drive,PITTSBURGH,PA,US,15275,UPS Next Day Air PM,40.449520,-80.179475
3,1441,SHANDS TEACHING HOSP & CL,2000 SOUTH WEST ARCHER ROAD,GAINESVILLE,FL,US,32610,NaN,29.681312,-82.353862
4,1442,ST LOUIS CHILDRENS HOSPITAL,4942 PARKVIEW PLACE,SAINT LOUIS,MO,US,63110,NaN,38.625401,-90.268132
...,...,...,...,...,...,...,...,...,...,...
2375,90640122,340B AHG of New York Inc DBA,125 Clearbrook Rd Ste 122,Elmsford,NY,US,10523,NaN,41.055100,-73.820130
2376,90640636,Life Science Logistics,302 33rd Street SE,Puyallup,WA,US,98372,NaN,47.205648,-122.266339
2377,90640637,Life Science Logistics,1190 Kennedy Rd,Windsor,CT,US,06095,NaN,41.871037,-72.675083
2378,90640638,Integrated Quality Solutions,3375 Koapaka St Suite A125,Honolulu,HI,US,96819,NaN,NaN,NaN


Appendix

In [14]:
# locator = Nominatim(user_agent="TS@MIT", timeout=10)
# geocode = RateLimiter(lambda query: locator.geocode(query, country_codes='us'), min_delay_seconds=1)

# # Test with US ZIP code
# print(geocode('20009'))

# # from geopy.geocoders import Nominatim
# # from geopy.extra.rate_limiter import RateLimiter
# # from geopy.exc import GeocoderTimedOut
# from tqdm import tqdm
# # import pandas as pd

# # Set up tqdm for pandas apply
# tqdm.pandas()

# # Initialize geolocator with US country restriction
# geolocator = Nominatim(user_agent="TS@MIT")
# rate_limited_geocode = RateLimiter(lambda z: geolocator.geocode(z, country_codes='us'), min_delay_seconds=1)

# # ZIP cache to avoid redundant geocoding
# zip_cache = {}

# # Function to geocode ZIP with caching
# def geocode_zip(zipcode):
#     zip_str = str(zipcode)[:5]
#     if zip_str in zip_cache:
#         return zip_cache[zip_str]
#     try:
#         location = rate_limited_geocode(zip_str)
#         if location:
#             lat_lon = (location.latitude, location.longitude)
#         else:
#             lat_lon = (None, None)
#     except GeocoderTimedOut:
#         return geocode_zip(zip_str)  # Retry on timeout
#     except Exception as e:
#         print(f"Error for ZIP {zip_str}: {e}")
#         lat_lon = (None, None)
#     zip_cache[zip_str] = lat_lon
#     return lat_lon

# # Clean ZIPs
# m_customers['zipcode'] = m_customers['zipcode'].astype(str).str[:5]

# # Apply with progress bar
# lat_lon_values = m_customers['zipcode'].progress_apply(geocode_zip)

# # Assign lat/lon back to DataFrame
# m_customers[['lat', 'lon']] = pd.DataFrame(lat_lon_values.tolist(), index=m_customers.index)

# # Success rate
# successful_geocodes = m_customers[['lat', 'lon']].notnull().all(axis=1).sum()
# total_addresses = len(m_customers)
# percentage_success = (successful_geocodes / total_addresses) * 100 if total_addresses > 0 else 0
# print(f"Percentage of successfully geocoded ZIPs: {percentage_success:.2f}%")

# # Preview results
# print(m_customers[['customer_id', 'zipcode', 'lat', 'lon']].head())